### 5.1 ベイズニューラルネットワークモデルの近似推論法  
バッチ学習によるニューラルネットワークの基本的な学習手法  
入力データ$X = \{x_1, ..., x_N\}$  
観測データ$Y=\{y_1,...,y_N\}$  
パラメータ$W$
$$
p(y, W|X) = p(W) \prod_{n=1}^N p(y_n |x_n, W) \tag{1}
$$
とおく  
$x_n\in \mathbb{R}^{H_0}$から$y_n \in \mathbb{R}^D$を予測する

$$
p(y_n|x_n, W) = N(y_n|f(x_n;W), \sigma^2_yI) \tag{2}
$$

$f(x_n;W)$がニューラルネットワーク  
ここでは分散$\sigma^2_y$は固定のノイズ項
$$
f(x_n;W) = \sum_{h_1=1}^{H_1} w_{d,h1}^{(2)}\phi \left( \sum_{h_0=1}^{H_0} w_{h_1, h_0}^{(1)} x_{n,h_0} \right) \tag{3}
$$

学習データが与えられた後のパラメータの事後分布をべいず推論で計算するのでパラメータに事前分布を設定する必要がある  
ここでは各重みパラメータを$w\in W$とし，それぞれに独立なガウス分布を与える  
$$
p(w) = N(w|0,\sigma_w^2) \tag{4}
$$

#### 5.1.2 ラプラス近似による学習  
式(1)のニューラルネットワークモデルに対するラプラス近似による学習と予測を導出する  
簡単のためD=1とする

1. モデルの事後分布のMAP推定値を最適化により求める　　
$$
p(W|Y,X) \propto p(W)p(Y|X,W) \tag{5}\\
$$
局所最適解$W_{MAP}$の更新は対数事後分布の勾配を利用して
$$
W_{new} = W_{old} + \alpha \nabla _w \log P(W|Y, W)|_{W=W_{old}} \tag{6}
$$
2. 周辺をガウス分布によって求める  


パラメータ$w\in W$の偏微分を計算すると   
$$
\frac{\partial}{\partial w} \log p(W|Y,X) = - \left\{ \frac{1}{\sigma_y^2}\frac{\partial}{\partial w} E(W) + \frac{1}{\sigma_w^2}\frac{\partial}{\partial w} \Omega_{L2} \right\} \tag{8}
$$

$\Omega_{L2}$は正規化項で，容易に微分できる  
$E(W)$はニューラルネットワークの誤差関数  

ラプラス近似で計算される近似が近似事後分布を
$$
q(W) = N(W|W_{MAP}, \{\Lambda (W_{MAP})\}^{-1}) \tag{9}
$$
とおく  
精度行列$\Lambda$は
\begin{eqnarray}
    \Lambda = -\nabla^2_W \log p(W|Y,X) \\
    = \frac{1}{\sigma_w^2} I + \frac{1}{\sigma_y^2}H \tag{10}
\end{eqnarray}
$H$はヘッセ行列で厳密計算あるいは近似計算によって求められる  

#### 5.1.2.2. 予測分布の近似  
$$
p(y_*|x_*, Y, X) \approx  \int p(y_* | x_*, W) q(W)dW \tag{11}
$$

パラメータWはq(W)としてガウス分布で近似しているが$p(y_*|x_*, W)$の中にニューラルネットワークが含まれているので解析的に計算できない　　
→ニューラルネットワークの線形近似を行う  
パラメータの事後分布の密度がMAP推定値の周辺に集中していて，その範囲においてニューラルネットワークの間数値$f(x_*, W)$がWの線形関数でよく近似できるという仮定をおく  

\begin{eqnarray}
f(x_* ; W) \approx f(x_* ; W_{MAP}) + g^T(W-W_{MAP}) \tag{12} \\
g = \nabla wf(x_*;W)|_{W=W_{MAP}} \tag{13}
\end{eqnarray}
テイラー展開でWの関数$f(x_*;W)$を$W_{MAP}$周りで一次近似したもの  
gは関数の勾配を$W_{MAP}$で評価したもの  
この近似を使えばニューラルネットワークの非線形性がなくなるので解析的に計算できるようになる  

\begin{eqnarray}
    p(y_*|x_*, Y, X ) \approx \int p(y_* | x_*, W)q(W)d(W) \\
    \approx \int N(y_*|f(x_* ; W_{MAP} ) + g^T(W - W_{MAP} ), \sigma^2_y)N(W|W_{MAP}, \{\Lambda (W_{MAP})\}^{-1})dW\\
    = N(y_* | f(x_*; W_{MAP}), \sigma^2(x_*)) \tag{14} \\
    ただし \sigma^2(x_*) = \sigma^2_y + g^T\{\Lambda (W_{MAP})\}^{-1}g
\end{eqnarray}


#### 5.1.3 ハミルトニアンモンテカルロ法による学習  
ハミルトニアンモンテカルロ法を利用したベイズミューラルネットワークの事後分布からのサンプリング  


ハミルトニアンモンテカルロ法は対数事後分布がサンプリングしたい変数に関して微分可能であれば適用できる  
離散変数をパラメータとしてもたないニューラルネットワークではハミルトニアンモンテカルロ法に必要な微分情報の計算には誤差逆伝播ほうが使える  
計算時間さえ十分に確保していれば理論的には真の事後分布からのサンプルが得られる  

式(5)の正規化されていない事後分布を利用すると対応するポテンシャルエネルギーは
$$
U(W) = - \{\log p(Y|X,W) + \log p(W)\} \tag{16}
$$

リープフロッグ法を使うためにはポテンシャルエネルギーの微分が必要  
→正則化項を付け加えたコスト関数の微分と等価なので誤差逆伝播法による勾配計算が利用できる  
ハミルトニアンモンテカルロ法では予測の不確実性がサンプリングに基づく手法で得られる複数の関数のサンプリングから表現される  

#### 5.1.3.2 ハイパーパラメータの推論
重みパラメータWの事前分布を支配する$\sigma^2_w$や観測モデルのノイズパラメータ$\sigma^2_y$などのハイパーパラメータに対して事前分布を設定する  
パラメータWは分散$\sigma^2_w$を持つガウス分布に従って決定される
$$
p(\gamma_w) = Gam(\gamma_w|a_w,b_w) \tag{17} \\
ただし\gamma_w = \sigma^{-2}_w
$$

観測ノイズも式(17)同様にガンマ分布を設定する  
これらの精度パラメータの事前分布を導入した場合のモデルは  
$$
p(Y,W,\gamma_w, \gamma_y | X) = p(\gamma_w)p(\gamma_y)p(W|\gamma_w) \prod_{n=1}^N p(y_n|x_n, W, \gamma_y) \tag{19}
$$
従って事後分布全体は
$$
p(W,\gamma_w, \gamma_y|Y,X) \tag{20}
$$となる

求めたい確率変数($W, \gamma_w, \gamma_y$)を条件付き確立を用いて別々にサンプリングする  
→他の２変数を既知として求めたいパラメータのみを確率変数として扱う  
$$
p(W|T,X, \gamma_w, \gamma_y) \tag{21}
$$
これはパラメータの事後分布そのままなのでハミルトニアンモンテカルロ法を実行することでWのサンプルを得られる  
$$
p(\gamma_w|Y,X,W,\gamma_y) \propto p(W|\gamma_w)p(\gamma_w) \tag{22}
$$
$p(W|\gamma_w)$はガウス分布，$\gamma_w$の事前分布はガンマ分布なので22式もガンマ分布として解析的に求められる  
\begin{eqnarray}
\gamma_w ~ Gam(\hat{a}_w, \hat{b}_w) \tag{23}\\
\hat{a}_w = a_w + \frac{K_w}{2} \tag{24}\\
\hat{b}_w = b_w + \frac{1}{2} \sum_{w\in W} w^2 \tag{25}\\
ただし，K_wは重みパラメータWの総数
\end{eqnarray}

$$
p(\gamma_y|Y,X,W,\gamma_w) \propto p(\gamma_y) \prod_{n=1}^N p(y_n|x_n,W, \gamma_y) \tag{26}
$$

と計算できる  
観測モデル$p(Y|W,X,\gamma_y)$がガウス分布，$\gamma_y$の事前分布にはガンマ分布を用いているので解析的に分布の計算ができる 